In [18]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [19]:
dantri_url = 'https://dantri.com.vn/'
response = requests.get(dantri_url)
response.text

'<!DOCTYPE html>\r\n<html lang="vi" xmlns="http://www.w3.org/1999/xhtml">\r\n<head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="format-detection" content="telephone=no"><meta name="robots" content="index,follow,all,max-image-preview:large"><title>Tin tức Việt Nam và quốc tế nóng, nhanh, cập nhật 24h | Báo Dân trí</title><meta name="description" content="Đọc báo dantri - Tin tức mới nhất, Thông tin nhanh chính xác được cập nhật hàng giờ. báo nói đọc tin tức online Việt Nam Thế giới nóng nhất trong ngày, Kinh doanh Việc làm, Pháp luật Đời sống, Giáo dục Sức khỏe, Thể thao Giải trí, Công nghệ, Văn hóa Xã hội, Ôtô Xe máy, Tình yêu, Nhân ái, Bạn đọc"><meta name="keywords" content="dantri,báo điện tử dantri,thời tiết,tin tức,tin nhanh,thể thao,dự báo thời tiết,tin the thao,thể thao 24h,giá vàng hôm nay,tin the gioi,đọc báo,tin moi,giá vàng,thời tiết hôm nay,tin tức online,bóng đá việt nam,pháp luật,dự báo thời tiết hôm nay,tin tức 24 g

In [20]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="vi" xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="index,follow,all,max-image-preview:large" name="robots"/><title>Tin tức Việt Nam và quốc tế nóng, nhanh, cập nhật 24h | Báo Dân trí</title><meta content="Đọc báo dantri - Tin tức mới nhất, Thông tin nhanh chính xác được cập nhật hàng giờ. báo nói đọc tin tức online Việt Nam Thế giới nóng nhất trong ngày, Kinh doanh Việc làm, Pháp luật Đời sống, Giáo dục Sức khỏe, Thể thao Giải trí, Công nghệ, Văn hóa Xã hội, Ôtô Xe máy, Tình yêu, Nhân ái, Bạn đọc" name="description"/><meta content="dantri,báo điện tử dantri,thời tiết,tin tức,tin nhanh,thể thao,dự báo thời tiết,tin the thao,thể thao 24h,giá vàng hôm nay,tin the gioi,đọc báo,tin moi,giá vàng,thời tiết hôm nay,tin tức online,bóng đá việt nam,pháp luật,dự báo thời tiết hôm nay,tin tức 24 giờ,tin nong,thời 

# Tạo danh sách các links để truy cập

In [21]:
# Lấy danh sách title + link
articles = []
for a in soup.select("article.article-item h3.article-title a"):
    title = a.get_text(strip=True)
    link = a["href"]
    articles.append({"title": title, "link": link})

print(len(articles))
print(articles[:5])

146
[{'title': 'Bão Matmo di chuyển nhanh, liên tục tăng cấp', 'link': 'https://dantri.com.vn/thoi-su/bao-matmo-di-chuyen-nhanh-lien-tuc-tang-cap-20251002205513642.htm'}, {'title': 'CLB Công an Hà Nội vươn lên dẫn đầu bảng tại Cúp C2 châu Á', 'link': 'https://dantri.com.vn/the-thao/clb-cong-an-ha-noi-vuon-len-dan-dau-bang-tai-cup-c2-chau-a-20251002223426170.htm'}, {'title': 'TPHCM làm rõ vấn đề pháp lý trong dự án của Lotte tại Thủ Thiêm', 'link': 'https://dantri.com.vn/bat-dong-san/tphcm-lam-ro-van-de-phap-ly-trong-du-an-cua-lotte-tai-thu-thiem-20251002190202241.htm'}, {'title': 'TPHCM ưu tiên đất công cộng làm trạm sạc xe điện', 'link': 'https://dantri.com.vn/thoi-su/tphcm-uu-tien-dat-cong-cong-lam-tram-sac-xe-dien-20251002202907272.htm'}, {'title': 'CLB Nam Định có chiến thắng thứ hai liên tiếp tại Cúp C2 châu Á', 'link': 'https://dantri.com.vn/the-thao/clb-nam-dinh-co-chien-thang-thu-hai-lien-tiep-tai-cup-c2-chau-a-20251002202152745.htm'}]


# Vào từng bài viết và lấy các thông tin cần thiết

- Phần này khá đơn giản do các thông tin này được load ngay khi truy cập vào trang.
- Sử dụng BS4 là đủ để crawl

In [22]:
def get_article_detail(url):
    try:
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")

        # Tiêu đề
        title_tag = soup.find("h1", class_="title-page")
        title = title_tag.get_text(strip=True) if title_tag else None

        # Sapo (đoan mở đầu)
        sapo_tag = soup.find("h2", class_="singular-sapo")
        sapo = sapo_tag.get_text(strip=True) if sapo_tag else None

        # Nội dung
        content_tag = soup.find("div", class_="singular-content")
        if content_tag:
            content = " ".join([p.get_text(strip=True) for p in content_tag.find_all("p")])
        else:
            content = None

        # Tác giả
        author_tag = soup.find("div", class_="author-name")
        author = author_tag.get_text(strip=True) if author_tag else None

        # Link của tác giả
        author_link = author_tag.a["href"] if author_tag and author_tag.a else None

        # Main section và Sub section
        # Tìm breadcrumb
        breadcrumb = soup.find("ul", class_="dt-text-c808080")

        links = breadcrumb.find_all("a")
        main_section = links[0].get_text(strip=True) if breadcrumb else None
        sub_section = links[1].get_text(strip=True) if (breadcrumb and len(breadcrumb) > 1) else None

        # Ngày đăng
        date_tag = soup.find("time", class_="author-time")
        post_date = date_tag.get_text(strip=True) if date_tag else None

        return {
            "link": url,
            "title": title,
            "sapo": sapo,
            "content": content,
            "author_name": author,
            "author_link": author_link,
            "main_section": main_section,
            "sub_section": sub_section,
            "post_date": post_date,
        }

    except Exception as e:
        # Nếu có bất kỳ lỗi nào (thường là trang không đúng định dạng chung)
        # In ra lỗi và trả về dict với toàn bộ trường = None
        print(f"Lỗi khi lấy {url}: {e}")
        return {
            "link": url,
            "title": None,
            "sapo": None,
            "content": None,
            "author_name": None,
            "author_link": None,
            "main_section": None,
            "sub_section": None,
            "post_date": None,
        }

In [23]:
# Trang bình thường
url = "https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm"
data = get_article_detail(url)
data


{'link': 'https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm',
 'title': 'Kình ngư đầu tiên của Việt Nam giành 2 HCV ở giải châu Á',
 'sapo': '(Dân trí) - Vận động viên (VĐV) bơi Nguyễn Huy Hoàng đã đi vào lịch sử khi trở thành kình ngư đầu tiên của Việt Nam giành 2 Huy chương vàng (HCV) ở giải châu Á khi về nhất nội dung 800m tự do nam vào tối 1/10.',
 'content': 'Kình ngư số một Việt Nam Nguyễn Huy Hoàng tiếp tục tỏa sáng rực rỡ tại giải vô địchthể thaodưới nước châu Á 2025. Tối nay (1/10), anh đã giành thêm HCV nội dung 800m tự do nam với thành tích 7 phút 57 giây 58. Huy Hoàng giành Huy chương vàng ở nội dung bơi 800m tại giải vô địch thể thao dưới nước châu Á 2025 (Ảnh chụp màn hình). Ở phần thi này, Huy Hoàng phải đối đầu với nhiều đối thủ mạnh như Rawat Kushagra (Ấn Độ), Khiew Hoe Yean (Malaysia), Xu Haibo (Trung Quốc), Sibirtsev Ilya (Uzbekistan) hay Tanaka Shun (Nhật Bản). Tuy nhiên, kình ngư quê Quảng Bình nhanh chóng

In [24]:
# Trang khác định dạng chung
invalid_url = "https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm"
invalid_data = get_article_detail(invalid_url)
invalid_data

Lỗi khi lấy https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm: 'NoneType' object has no attribute 'find_all'


{'link': 'https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm',
 'title': None,
 'sapo': None,
 'content': None,
 'author_name': None,
 'author_link': None,
 'main_section': None,
 'sub_section': None,
 'post_date': None}

# Cào lượng reactions và chi tiết reactions

- Do reactions thuộc dạng lazyload nên BS4 bình thường không thể đọc được.
- Cần dùng selenium để giả lập việc mở trang web, sau đó lăn chuột xuống giữa trang vì reactions
nằm ở giữa trang và cần khoảng 1-2s để load
- Nếu muốn lấy thêm data nằm ở cuối trang thì cần thêm việc lăn chuột tới cuối trang nữa
- Dantri là 1 trang báo lazyload khá nhiều phần

In [25]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time


In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_reactions(data: dict, url: str) -> dict:
    try:
        if data is None:
            data = {}

        driver = webdriver.Chrome()
        driver.get(url)

        # --- Scroll chậm rãi toàn trang ---
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        step = scroll_height // 10   # chia trang thành 10 phần
        for i in range(0, scroll_height, step):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(1.5)  # đợi load react nếu có
        # đảm bảo tới cuối
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, "html.parser")
        buttons = soup.find_all("button", {"aria-label": True})

        # Chỉ giữ 4 loại react cần thiết
        labels = {"Bổ ích", "Xúc động", "Cảm hứng", "Độc đáo"}

        total_reacts = 0
        detailed_reacts = {}

        for btn in buttons:
            label = btn.get("aria-label", "")
            if label not in labels:   # bỏ qua react ngoài
                continue

            count_text = btn.get_text(strip=True)
            try:
                count = int(count_text) if count_text.isdigit() else 0
            except:
                count = 0

            detailed_reacts[label] = count
            total_reacts += count

        data["total_reacts"] = total_reacts
        data["detailed_reacts"] = detailed_reacts

        return data

    except Exception as e:
        # Nếu có bất kỳ lỗi nào (thường là trang không đúng định dạng chung)
        # In ra lỗi và trả về dict với toàn bộ trường = None
        print(f"Lỗi khi lấy reactions của links: {url}: {e}")
        data["total_reacts"] = 0
        data["detailed_reacts"] = None
        return data



In [27]:
test = get_reactions(data, url)
test

{'link': 'https://dantri.com.vn/the-thao/kinh-ngu-dau-tien-cua-viet-nam-gianh-2-hcv-o-giai-chau-a-20251001232131492.htm',
 'title': 'Kình ngư đầu tiên của Việt Nam giành 2 HCV ở giải châu Á',
 'sapo': '(Dân trí) - Vận động viên (VĐV) bơi Nguyễn Huy Hoàng đã đi vào lịch sử khi trở thành kình ngư đầu tiên của Việt Nam giành 2 Huy chương vàng (HCV) ở giải châu Á khi về nhất nội dung 800m tự do nam vào tối 1/10.',
 'content': 'Kình ngư số một Việt Nam Nguyễn Huy Hoàng tiếp tục tỏa sáng rực rỡ tại giải vô địchthể thaodưới nước châu Á 2025. Tối nay (1/10), anh đã giành thêm HCV nội dung 800m tự do nam với thành tích 7 phút 57 giây 58. Huy Hoàng giành Huy chương vàng ở nội dung bơi 800m tại giải vô địch thể thao dưới nước châu Á 2025 (Ảnh chụp màn hình). Ở phần thi này, Huy Hoàng phải đối đầu với nhiều đối thủ mạnh như Rawat Kushagra (Ấn Độ), Khiew Hoe Yean (Malaysia), Xu Haibo (Trung Quốc), Sibirtsev Ilya (Uzbekistan) hay Tanaka Shun (Nhật Bản). Tuy nhiên, kình ngư quê Quảng Bình nhanh chóng

# Áp dụng 2 hàm trên 1 batch test

In [28]:
test = articles[:10]
test

[{'title': 'Bão Matmo di chuyển nhanh, liên tục tăng cấp',
  'link': 'https://dantri.com.vn/thoi-su/bao-matmo-di-chuyen-nhanh-lien-tuc-tang-cap-20251002205513642.htm'},
 {'title': 'CLB Công an Hà Nội vươn lên dẫn đầu bảng tại Cúp C2 châu Á',
  'link': 'https://dantri.com.vn/the-thao/clb-cong-an-ha-noi-vuon-len-dan-dau-bang-tai-cup-c2-chau-a-20251002223426170.htm'},
 {'title': 'TPHCM làm rõ vấn đề pháp lý trong dự án của Lotte tại Thủ Thiêm',
  'link': 'https://dantri.com.vn/bat-dong-san/tphcm-lam-ro-van-de-phap-ly-trong-du-an-cua-lotte-tai-thu-thiem-20251002190202241.htm'},
 {'title': 'TPHCM ưu tiên đất công cộng làm trạm sạc xe điện',
  'link': 'https://dantri.com.vn/thoi-su/tphcm-uu-tien-dat-cong-cong-lam-tram-sac-xe-dien-20251002202907272.htm'},
 {'title': 'CLB Nam Định có chiến thắng thứ hai liên tiếp tại Cúp C2 châu Á',
  'link': 'https://dantri.com.vn/the-thao/clb-nam-dinh-co-chien-thang-thu-hai-lien-tiep-tai-cup-c2-chau-a-20251002202152745.htm'},
 {'title': 'Thủ tướng: Hỗ trợ 2.

In [29]:
all_data = []

for art in test:
    data = get_article_detail(art["link"])      
    data = get_reactions(data, art["link"])       
    all_data.append(data)                        

Lỗi khi lấy https://dantri.com.vn/tam-diem/chat-dut-moi-quan-he-quan-tham-gian-thuong-20251002063026629.htm: 'NoneType' object has no attribute 'find_all'
Lỗi khi lấy https://dantri.com.vn/doi-song/canh-cong-an-vuot-lu-dua-be-gai-11-tuoi-di-cap-cuu-20251002185816056.htm: 'NoneType' object has no attribute 'find_all'
Lỗi khi lấy https://dantri.com.vn/bat-dong-san/hang-trieu-gia-dinh-an-cu-nho-quy-nha-o-xa-hoi-nguoi-viet-mong-som-co-nha-20251002125623821.htm: 'NoneType' object has no attribute 'find_all'


In [30]:
type(all_data)

list

In [31]:
all_data

[{'link': 'https://dantri.com.vn/thoi-su/bao-matmo-di-chuyen-nhanh-lien-tuc-tang-cap-20251002205513642.htm',
  'title': 'Bão Matmo di chuyển nhanh, liên tục tăng cấp',
  'sapo': '(Dân trí) - Dự báo trong 24 đến 48 giờ tới, bão Matmo di chuyển nhanh, liên tục tăng cấp. Đến tối 5/10, bão trên bán đảo Lôi Châu (Trung Quốc) với cường độ cấp 11-12, giật cấp 15.',
  'content': 'Trung tâm Dự báo khí tượng thủy văn quốc gia cho biết lúc 19h ngày 2/10, bão Matmo trên vùng biển phía Đông Philippines. Sức gió mạnh nhất vùng gần tâm bão cấp 8-9 (62-88km/h), giật cấp 11. Trong 24 giờ tới, bão di chuyển theo hướng Tây Tây Bắc, tốc độ khoảng 25km/h. Tối 3/10, bão trên vùng biển phía Tây đảo Lu Dông (Philippines) với sức gió cấp 9, giật cấp 11. Trong 48 giờ tiếp theo, bão Matmo di chuyển chủ yếu theo hướng Tây Tây Bắc với tốc độ khoảng 25km/h và liên tục mạnh lên. Đến 19h ngày 5/10, bão trên bán đảo Lôi Châu (Trung Quốc) với cường độ cấp 11-12, giật cấp 15. Từ 72 đến 120 giờ tiếp theo, bão di chuyển t

In [32]:
all_data_df = pd.DataFrame(all_data)
all_data_df.head()

,link,title,sapo,content,author_name,author_link,main_section,sub_section,post_date,total_reacts,detailed_reacts
0,https://dantri.com.vn/thoi-su/bao-matmo-di-chu...,"Bão Matmo di chuyển nhanh, liên tục tăng cấp","(Dân trí) - Dự báo trong 24 đến 48 giờ tới, bã...",Trung tâm Dự báo khí tượng thủy văn quốc gia c...,Nguyễn Hải,https://dantri.com.vn/tac-gia/nguyen-hai-3154.htm,Thời sự,None,"Thứ năm, 02/10/2025 - 21:09",7,"{'Bổ ích': 2, 'Xúc động': 4, 'Cảm hứng': 1, 'Đ..."
1,https://dantri.com.vn/the-thao/clb-cong-an-ha-...,CLB Công an Hà Nội vươn lên dẫn đầu bảng tại C...,(Dân trí) - Dễ dàng giành chiến thắng đậm 3-0 ...,Trận đấu giữa CLB CAHN và Tai Po diễn ra tối n...,Trọng Vũ,https://dantri.com.vn/tac-gia/trong-vu-143.htm,Thể thao,Bóng đá,"Thứ năm, 02/10/2025 - 22:36",0,"{'Bổ ích': 0, 'Xúc động': 0, 'Cảm hứng': 0, 'Đ..."
2,https://dantri.com.vn/bat-dong-san/tphcm-lam-r...,TPHCM làm rõ vấn đề pháp lý trong dự án của Lo...,(Dân trí) - Sở Nông nghiệp và Môi trường TPHCM...,Phát biểu tại buổi họp báo cung cấp thông tin ...,Xuân Đoàn,https://dantri.com.vn/tac-gia/xuan-doan-3489.htm,Bất động sản,Thị trường,"Thứ năm, 02/10/2025 - 22:58",0,"{'Bổ ích': 0, 'Xúc động': 0, 'Cảm hứng': 0, 'Đ..."
3,https://dantri.com.vn/thoi-su/tphcm-uu-tien-da...,TPHCM ưu tiên đất công cộng làm trạm sạc xe điện,"(Dân trí) - Theo chỉ đạo của Chủ tịch TPHCM, đ...",Văn phòng UBND TPHCM vừa thông báo kết luận củ...,Q.Huy,https://dantri.com.vn/tac-gia/qhuy-284.htm,Thời sự,None,"Thứ năm, 02/10/2025 - 20:55",3,"{'Bổ ích': 2, 'Xúc động': 1, 'Cảm hứng': 0, 'Đ..."
4,https://dantri.com.vn/the-thao/clb-nam-dinh-co...,CLB Nam Định có chiến thắng thứ hai liên tiếp ...,(Dân trí) - Vượt qua đội bóng Hong Kong (Trung...,Dù thi đấu trên sân đối phương trong trận đấu ...,Trọng Vũ,https://dantri.com.vn/tac-gia/trong-vu-143.htm,Thể thao,Bóng đá,"Thứ năm, 02/10/2025 - 20:26",2,"{'Bổ ích': 2, 'Xúc động': 0, 'Cảm hứng': 0, 'Đ..."


# Áp dụng trên toàn bộ các bài báo